# Introduction to Autogen AgentChat

In [ ]:
%pip install "autogen-core"
%pip install -U "autogen-agentchat"
%pip install "autogen-ext[openai]"

In [3]:
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("Async environment configured for Jupyter.")
except ImportError:
    print("Please install nest_asyncio with `pip install nest_asyncio`")

Async environment configured for Jupyter.


In [4]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
import os
import asyncio

# Simple Agent

In [3]:
import asyncio
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)

    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant that helps students \
            understand how to build AI Agents! You always greet students \
                by saing 'Welcome to this Course about Autogen!'",
        model_client=model_client,
    )

    cancellation_token = CancellationToken()
    response = await assistant.on_messages([TextMessage(content="Hello!", source="user")], cancellation_token)
    print(response)

# This allows running async code in Jupyter

asyncio.run(main())

Response(chat_message=TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=47, completion_tokens=21), content='Welcome to this Course about Autogen! How can I assist you today in learning about AI agents?', type='TextMessage'), inner_messages=[])


In [21]:
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", seed=42, temperature=0)

assistant_helper = AssistantAgent(
    name="assistant_helper2",
    system_message = "You are a helpful assistant",
    model_client=model_client,
)

cancellation_token = CancellationToken()

In [22]:
async def run_my_assistant():
    response = await assistant_helper.on_messages([TextMessage(content="Hello!", source="user")], cancellation_token)
    print(response)

asyncio.run(run_my_assistant())

Response(chat_message=TextMessage(source='assistant_helper2', models_usage=RequestUsage(prompt_tokens=19, completion_tokens=10), content='Hello! How can I assist you today?', type='TextMessage'), inner_messages=[])


# Multimodal Agent

In [23]:
import asyncio
from pathlib import Path
from autogen_agentchat.messages import MultiModalMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken, Image
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)

    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant.",
        model_client=model_client,
    )

    cancellation_token = CancellationToken()
    message = MultiModalMessage(
        content=["Here is an image:", Image.from_file(Path("test.png"))],
        source="user",
    )
    response = await assistant.on_messages([message], cancellation_token)
    print(response)

asyncio.run(main())

Response(chat_message=TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=891, completion_tokens=81), content='The image is a webpage for "AgentChat," a high-level API for building multi-agent applications. It is built on top of the `autogen-core` package. The page highlights features like intuitive defaults, agents with preset behaviors, and teams with predefined multi-agent design patterns. It includes sections for Installation, Quickstart, Tutorial, and Selector Group Chat, each offering guidance on using AgentChat.', type='TextMessage'), inner_messages=[])


# Adding Tools

In [24]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core import CancellationToken

async def fetch_weather(city: str) -> str:
    return f"The weather in {city} is sunny."

# Initialize the agent
model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)
assistant = AssistantAgent(
    name="assistant",
    model_client=model_client,
    system_message="You return weather information given a city name using the fetch_weather tool.",
    tools=[fetch_weather], # Register the tool function
    )



output = await assistant.on_messages([TextMessage(content="What is the weather in San Francisco?", source="user")], CancellationToken())

In [25]:
for msg in output.inner_messages:
    print(msg.content)

[FunctionCall(id='call_xncCDqs7ZHUp24YdaTGqEsjT', arguments='{"city":"San Francisco"}', name='fetch_weather')]
[FunctionExecutionResult(content='The weather in San Francisco is sunny.', call_id='call_xncCDqs7ZHUp24YdaTGqEsjT')]


In [26]:
output.chat_message.content

'The weather in San Francisco is sunny.'

In [56]:
from autogen_core.tools import FunctionTool


#!pip install yfinance matplotlib pytz numpy pandas python-dotenv requests bs4
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:  # type: ignore[type-arg]
    import os
    import time

    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    api_key = os.getenv("GOOGLE_API_KEY")
    search_engine_id = os.getenv("GOOGLE_CSE_ID")

    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://customsearch.googleapis.com/customsearch/v1"
    params = {"key": str(api_key), "cx": str(search_engine_id), "q": str(query), "num": str(num_results)}

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("items", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append(
            {"title": item["title"], "link": item["link"], "snippet": item["snippet"], "body": body}
        )
        time.sleep(1)  # Be respectful to the servers

    return enriched_results

search_tool = FunctionTool(
    google_search, description="Search Google for information, returns results with a snippet and body content"
)

In [57]:
search_agent = AssistantAgent(
    name="Google_Search_Agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    tools=[search_tool],
    description="Search Google for information, returns top 5 results with a snippet and body content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

In [62]:
response = await search_agent.on_messages([TextMessage(content="How to set up custom tools in Autogen Agentchat version 0.4.1?", source="user")], CancellationToken())

In [63]:
response.chat_message.content

"[{'title': 'microsoft/autogen: A programming framework for agentic AI ... - GitHub', 'link': 'https://github.com/microsoft/autogen', 'snippet': '# pip install -U autogen-agentchat autogen-ext[openai,web-surfer] # playwright install ... You can use the AutoGen framework and developer tools to create\\xa0...', 'body': 'GitHub - microsoft/autogen: A programming framework for agentic AI 🤖 PyPi: autogen-agentchat Discord: https://aka.ms/autogen-discord Office Hour: https://aka.ms/autogen-officehour Skip to content Navigation Menu Toggle navigation Sign in Product GitHub Copilot Write better code with AI Security Find and fix vulnerabilities Actions Automate any workflow Codespaces Instant dev environments Issues Plan and track work Code Review Manage code changes Discussions Collaborate outside of code Code'}, {'title': 'AutoGen', 'link': 'https://www.reddit.com/r/AutoGenAI/', 'snippet': 'Sep 27, 2023 ... Improved component config by allowing subclassing the BaseComponent class. #5017. To 

In [64]:
from ast import literal_eval
results = literal_eval(response.chat_message.content)

In [65]:
results

[{'title': 'microsoft/autogen: A programming framework for agentic AI ... - GitHub',
  'link': 'https://github.com/microsoft/autogen',
  'snippet': '# pip install -U autogen-agentchat autogen-ext[openai,web-surfer] # playwright install ... You can use the AutoGen framework and developer tools to create\xa0...',
  'body': 'GitHub - microsoft/autogen: A programming framework for agentic AI 🤖 PyPi: autogen-agentchat Discord: https://aka.ms/autogen-discord Office Hour: https://aka.ms/autogen-officehour Skip to content Navigation Menu Toggle navigation Sign in Product GitHub Copilot Write better code with AI Security Find and fix vulnerabilities Actions Automate any workflow Codespaces Instant dev environments Issues Plan and track work Code Review Manage code changes Discussions Collaborate outside of code Code'},
 {'title': 'AutoGen',
  'link': 'https://www.reddit.com/r/AutoGenAI/',
  'snippet': 'Sep 27, 2023 ... Improved component config by allowing subclassing the BaseComponent class. #

In [66]:
# Loop through and print each result nicely formatted
for result in results:
    print("Title:", result['title'])
    print("Link:", result['link'])
    print("Snippet:", result['snippet'])
    print("Body:", result['body'][:200] + "...")  # Print first 200 chars of body
    print("-" * 80)  # Print separator line

Title: microsoft/autogen: A programming framework for agentic AI ... - GitHub
Link: https://github.com/microsoft/autogen
Snippet: # pip install -U autogen-agentchat autogen-ext[openai,web-surfer] # playwright install ... You can use the AutoGen framework and developer tools to create ...
Body: GitHub - microsoft/autogen: A programming framework for agentic AI 🤖 PyPi: autogen-agentchat Discord: https://aka.ms/autogen-discord Office Hour: https://aka.ms/autogen-officehour Skip to content Navi...
--------------------------------------------------------------------------------
Title: AutoGen
Link: https://www.reddit.com/r/AutoGenAI/
Snippet: Sep 27, 2023 ... Improved component config by allowing subclassing the BaseComponent class. #5017. To read more about how to create your own component config to ...
Body: Reddit - Dive into anything Skip to main content We value your privacy Reddit and its partners use cookies and similar technologies to provide you with a better experience. By accepti

Cancellation tokenn is something you use to cancell the async calls during an agent's execution.

# What is Cancellation Token?

Token to cancel pending async calls

In [ ]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main():
    # Initialize the model client
    model_client = OpenAIChatCompletionClient(model="gpt-4o")

    # Create an instance of AssistantAgent
    agent = AssistantAgent(name="assistant", model_client=model_client)

    # Create a cancellation token
    cancellation_token = CancellationToken()

    try:
        # Start a task that will cancel the token after 5 seconds
        asyncio.create_task(cancel_after_delay(cancellation_token, 3))

        # Send a message to the agent with the cancellation token
        response = await agent.on_messages(
            [TextMessage(content="Write me a story about the history of Large Language Models", source="user")],
            cancellation_token=cancellation_token,
        )
        print(response.chat_message.content)
    except asyncio.CancelledError:
        print("The operation was cancelled.")

async def cancel_after_delay(cancellation_token, delay):
    await asyncio.sleep(delay)
    cancellation_token.cancel()

# Run the main function
asyncio.run(main())


The operation was cancelled.


Cancellation tokenn is something you use to cancell the async calls during an agent's execution.